# DATA EXTRACTION
# Scraping brands from goodonyou.eco

https://directory.goodonyou.eco/categories/tops

### 0. Import modules

In [2]:
import pickle
import json

import pandas as pd
import time
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')


from selenium import webdriver
PATH = webdriver.FirefoxOptions() 


from selenium import webdriver
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait   # para esperar a que cargue
#from selenium.webdriver.support import expected_conditions as EC   # condiciones esperadas


PATH = webdriver.FirefoxOptions() 

## 0. Beautiful soup

In [5]:
import requests as req
from bs4 import BeautifulSoup as bs   # ambos alias son cosa mia

In [6]:
url = 'https://directory.goodonyou.eco/categories/tops'

In [9]:
# usamos requests para extraer el html

html = req.get(url).content    # o .text

soup = bs(html, 'html.parser')

type(soup)

bs4.BeautifulSoup

In [ ]:
soup.find('div', id='geoResult')

/html/body/div/div/div[4]/div/div[2]/div/div

In [24]:
tabla = soup.find('div', {'class': 'ResponsiveGrid__GridRow-sc-1sitd6c-0 hotiPS'})

In [37]:
tabla

<div class="ResponsiveGrid__GridRow-sc-1sitd6c-0 hotiPS"><div class="ResponsiveGrid__GridItem-sc-1sitd6c-1 jOebYD"><div class="BrandCard__Wrapper-sc-295tij-2 dksYSl"><div class="StyledBox-sc-13pk1d4-0 gttMNt"><div class="StyledBox-sc-13pk1d4-0 hYhslw BrandCard__ImageWrapper-sc-295tij-0 kregWb"><a class="Link__LinkBody-sc-o1cqd2-1 ijSNSa" href="/brand/thunderpants-nz"><span style="box-sizing:border-box;display:block;overflow:hidden;width:initial;height:initial;background:none;opacity:1;border:0;margin:0;padding:0;position:absolute;top:0;left:0;bottom:0;right:0"><noscript><img alt="related brand image" class="BrandCard__StyledImage-sc-295tij-1 bGWRzT" data-nimg="fill" decoding="async" loading="lazy" sizes="100vw" src="/_next/image?url=https%3A%2F%2Fd1mrofrbtdll1g.cloudfront.net%2Fed%2Fbrands%2Fimage%2F2020%2F71%2F1597199604539.jpeg&amp;w=3840&amp;q=75" srcset="/_next/image?url=https%3A%2F%2Fd1mrofrbtdll1g.cloudfront.net%2Fed%2Fbrands%2Fimage%2F2020%2F71%2F1597199604539.jpeg&amp;w=640&amp

In [35]:
tabla_tags = tabla.find_all('div', {'class': 'StyledBox-sc-13pk1d4-0 qkiiL'})

In [36]:
len(tabla_tags)

0

## 1. Use Selenium to scrap the data

### 1.1  Open a Firefox window - driver

In [3]:
#driver.quit()
driver = webdriver.Firefox(options = PATH)
#driver.quit()

### 1.2 Go to: `www.amazon.es`

In [4]:
url = 'https://directory.goodonyou.eco/categories/tops'
driver.get(url)

###  1.3 Accept cookies if asked

In [10]:
try:
    ## if amazon ask for cookies first
    try:
        driver.find_element(By.XPATH, '//*[@id="sp-cc-accept"]').click()

    ## if not, go directly to departments, there amazon will surely ask for cookies
    except:
        driver.find_element(By.XPATH, '/html/body/div/div[1]/div/div[1]/a[2]').click()
        time.sleep(3) # espera 2 segundos
        driver.find_element(By.XPATH, '/html/body/center/table/tbody/tr/td[2]/b[2]/a').click()
        ## cookies
        time.sleep(3) # espera 2 segundos
        driver.find_element(By.XPATH, '//*[@id="sp-cc-accept"]').click()
except:
    print("other new starting page")

other new starting page


### 1.4 Navigate the web "by-clicking" to get `women t-shirts`

Hamburguer button > ver todo > ropa > mujer > ropa > camiseta

In [11]:
driver.find_element(By.XPATH, '//*[@id="nav-hamburger-menu"]').click()
time.sleep(4) # espera 2 segundos
driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div/ul[1]/li[21]/a[1]').click()
time.sleep(4) # espera 2 segundos
driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/div/ul[1]/ul[1]/li[7]/a').click()
time.sleep(4) # espera 2 segundos

Select the path you want, either men or women

In [12]:
men='/html/body/div[3]/div[2]/div/ul[44]/li[5]/a'
women='/html/body/div[3]/div[2]/div/ul[44]/li[4]/a'

driver.find_element(By.XPATH, men).click() ####
time.sleep(4) # espera 2 segundos


In [15]:
#clothes
driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/ul/li[8]/span/a/span').click()
time.sleep(4) # espera 2 segundos
#tshirt
driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/ul/li[6]/span/a/span').click()
time.sleep(4) # espera 2 segundos
#driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/ul/li[6]/span/a/span').click()

# ver todos los resultados
driver.find_element(By.XPATH,'//*[@id="apb-desktop-browse-search-see-all"]').click()
time.sleep(4)

start_path=driver.current_url #save the url of the origin page before selecting colors

### 1.5 Scrap all the content related to the products `by color` and  `by page`


* To move through the colors, I need to click the color, scrap and come back to the starting url. <br/>

* To move along pages within each color, in need to pick a random number between 7-10.  <br/>

* Once I get 20 pages for each of the 16 colors available (left-centered box), the loop ends and saves the data in a dictionary of lists.

* Each time I scrap the data of a page, I need to parse it to text before moving to other urls.

`sel_to_text` function parses selenium object to text and filters out irreleant elements (4 < len < 10) 


In [16]:
def sel_to_text(selenium_list, min_=4, max_=10):
    text_list=[prod.text.split("\n") for prod in selenium_list] # selenium link to text
    text_list = [prod for prod in text_list if min_ < len(prod) < max_] # filter out irrelevant elements
    return text_list

**Don't run it**, super slow code. <br/>
The same code **works for either men or women** clothes, just uncomment the code lines as necessary

In [19]:
driver.get(start_path)

color_index=range(1,16)

index_list=[7,8,9,10]*30

PAG_COLOR = 20 # define total pages for each color

products_pag_dict = {}

for c in color_index:

    driver.get(start_path)
    #men
    color_path=f'/html/body/div[1]/div[1]/div[1]/div[2]/div/div[3]/span/div[1]/div/div/div[9]/ul[4]/span[{str(c)}]/li/span/a'
    #women
    #color_path=f'/html/body/div[1]/div[1]/div[1]/div[2]/div/div[3]/span/div[1]/div/div/div[8]/ul[4]/span[{str(c)}]/li/span/a'
    test_col=driver.find_elements(By.XPATH, color_path)

    color_value=test_col[0].get_attribute("title")

    color_url=test_col[0].get_attribute("href") # url to the (next) color
    driver.get(color_url)

    print(f"Start with Color: {color_value}")
    time.sleep(5) # espera 5 segundos


    #### # each page of each color - FIRST PAGE
    count=1

    products_pag_list=[] 
    test0 = driver.find_elements(By.CLASS_NAME, 'sg-col-inner') # GET data FIRST PAGE
    #test0 = driver.find_elements(By.CLASS_NAME, 'a-section a-spacing-small puis-padding-left-micro puis-padding-right-micro') # GET data 
    TT=sel_to_text(test0)
    products_pag_list.append(TT) # FUN: append the relevant elements into the growing list

        #### # each page of each color - FOLLOWING PAGES
    for index in index_list: # try random "nth-child" indexes

        nextpag=f'a.s-pagination-item:nth-child({str(index)})'

        try: 
            driver.find_element(By.CSS_SELECTOR, nextpag).click()  #CHANGE page /within color X
            time.sleep(4) # espera 4 segundos
            try: 
                test0 = driver.find_elements(By.CLASS_NAME, 'sg-col-inner') # GET data FIRST PAGE
                TT=sel_to_text(test0)
                products_pag_list.append(TT) # FUN: append the relevant elements into the growing list

                time.sleep(2) # espera 2 segundos
                    #
                count += 1
                print("y", index)

            except:
                print("-", index)    
        except:
            print("n", index)
            #pass

        if count <= PAG_COLOR :
            print(f"Count: {count},  Continue {color_value}") #############

        else: # get info from 20 pages - STOP and GO to other color
            products_pag_dict[color_value] = products_pag_list # add the color list to a dict

            print("Color done")
            driver.get(start_path) # back to starting link to select a new color
            time.sleep(5) # espera 5 segundos
            break
            print("shouldn't be printed")

    print(f"Count: {count},  Next color")

print("All done!!")    

Start with Color: Negro
n 7
Count: 1,  Continue Negro
y 8
Count: 2,  Continue Negro
n 9
Count: 2,  Continue Negro
n 10
Count: 2,  Continue Negro
y 7
Count: 3,  Continue Negro
y 8
Count: 4,  Continue Negro
y 9
Count: 5,  Continue Negro
n 10
Count: 5,  Continue Negro
n 7
Count: 5,  Continue Negro
n 8
Count: 5,  Continue Negro
y 9
Count: 6,  Continue Negro
n 10
Count: 6,  Continue Negro
n 7
Count: 6,  Continue Negro
n 8
Count: 6,  Continue Negro
y 9
Count: 7,  Continue Negro
n 10
Count: 7,  Continue Negro
n 7
Count: 7,  Continue Negro
n 8
Count: 7,  Continue Negro
y 9
Count: 8,  Continue Negro
n 10
Count: 8,  Continue Negro
n 7
Count: 8,  Continue Negro
n 8
Count: 8,  Continue Negro
y 9
Count: 9,  Continue Negro
n 10
Count: 9,  Continue Negro
n 7
Count: 9,  Continue Negro
n 8
Count: 9,  Continue Negro
y 9
Count: 10,  Continue Negro
n 10
Count: 10,  Continue Negro
n 7
Count: 10,  Continue Negro
n 8
Count: 10,  Continue Negro
y 9
Count: 11,  Continue Negro
n 10
Count: 11,  Continue Negro
n 

y 9
Count: 16,  Continue Marrón
n 10
Count: 16,  Continue Marrón
n 7
Count: 16,  Continue Marrón
n 8
Count: 16,  Continue Marrón
y 9
Count: 17,  Continue Marrón
n 10
Count: 17,  Continue Marrón
n 7
Count: 17,  Continue Marrón
n 8
Count: 17,  Continue Marrón
y 9
Count: 18,  Continue Marrón
n 10
Count: 18,  Continue Marrón
n 7
Count: 18,  Continue Marrón
n 8
Count: 18,  Continue Marrón
y 9
Count: 19,  Continue Marrón
n 10
Count: 19,  Continue Marrón
n 7
Count: 19,  Continue Marrón
n 8
Count: 19,  Continue Marrón
y 9
Count: 20,  Continue Marrón
n 10
Count: 20,  Continue Marrón
n 7
Count: 20,  Continue Marrón
n 8
Count: 20,  Continue Marrón
y 9
Color done
Count: 21,  Next color
Start with Color: Beis
n 7
Count: 1,  Continue Beis
y 8
Count: 2,  Continue Beis
n 9
Count: 2,  Continue Beis
n 10
Count: 2,  Continue Beis
y 7
Count: 3,  Continue Beis
y 8
Count: 4,  Continue Beis
y 9
Count: 5,  Continue Beis
n 10
Count: 5,  Continue Beis
n 7
Count: 5,  Continue Beis
n 8
Count: 5,  Continue Beis
y 

y 9
Count: 12,  Continue Naranja
n 10
Count: 12,  Continue Naranja
n 7
Count: 12,  Continue Naranja
n 8
Count: 12,  Continue Naranja
y 9
Count: 13,  Continue Naranja
n 10
Count: 13,  Continue Naranja
n 7
Count: 13,  Continue Naranja
n 8
Count: 13,  Continue Naranja
y 9
Count: 14,  Continue Naranja
n 10
Count: 14,  Continue Naranja
n 7
Count: 14,  Continue Naranja
n 8
Count: 14,  Continue Naranja
y 9
Count: 15,  Continue Naranja
n 10
Count: 15,  Continue Naranja
n 7
Count: 15,  Continue Naranja
n 8
Count: 15,  Continue Naranja
y 9
Count: 16,  Continue Naranja
n 10
Count: 16,  Continue Naranja
n 7
Count: 16,  Continue Naranja
n 8
Count: 16,  Continue Naranja
y 9
Count: 17,  Continue Naranja
n 10
Count: 17,  Continue Naranja
n 7
Count: 17,  Continue Naranja
n 8
Count: 17,  Continue Naranja
y 9
Count: 18,  Continue Naranja
n 10
Count: 18,  Continue Naranja
n 7
Count: 18,  Continue Naranja
n 8
Count: 18,  Continue Naranja
y 9
Count: 19,  Continue Naranja
n 10
Count: 19,  Continue Naranja
n 

n 7
Count: 1,  Continue Turquesa
y 8
Count: 2,  Continue Turquesa
n 9
Count: 2,  Continue Turquesa
n 10
Count: 2,  Continue Turquesa
y 7
Count: 3,  Continue Turquesa
y 8
Count: 4,  Continue Turquesa
y 9
Count: 5,  Continue Turquesa
n 10
Count: 5,  Continue Turquesa
n 7
Count: 5,  Continue Turquesa
n 8
Count: 5,  Continue Turquesa
y 9
Count: 6,  Continue Turquesa
n 10
Count: 6,  Continue Turquesa
n 7
Count: 6,  Continue Turquesa
n 8
Count: 6,  Continue Turquesa
y 9
Count: 7,  Continue Turquesa
n 10
Count: 7,  Continue Turquesa
n 7
Count: 7,  Continue Turquesa
n 8
Count: 7,  Continue Turquesa
y 9
Count: 8,  Continue Turquesa
n 10
Count: 8,  Continue Turquesa
n 7
Count: 8,  Continue Turquesa
n 8
Count: 8,  Continue Turquesa
y 9
Count: 9,  Continue Turquesa
n 10
Count: 9,  Continue Turquesa
n 7
Count: 9,  Continue Turquesa
n 8
Count: 9,  Continue Turquesa
y 9
Count: 10,  Continue Turquesa
n 10
Count: 10,  Continue Turquesa
n 7
Count: 10,  Continue Turquesa
n 8
Count: 10,  Continue Turquesa

y 9
Count: 14,  Continue Dorado
n 10
Count: 14,  Continue Dorado
n 7
Count: 14,  Continue Dorado
n 8
Count: 14,  Continue Dorado
y 9
Count: 15,  Continue Dorado
n 10
Count: 15,  Continue Dorado
n 7
Count: 15,  Continue Dorado
n 8
Count: 15,  Continue Dorado
y 9
Count: 16,  Continue Dorado
n 10
Count: 16,  Continue Dorado
n 7
Count: 16,  Continue Dorado
n 8
Count: 16,  Continue Dorado
y 9
Count: 17,  Continue Dorado
n 10
Count: 17,  Continue Dorado
n 7
Count: 17,  Continue Dorado
n 8
Count: 17,  Continue Dorado
y 9
Count: 18,  Continue Dorado
n 10
Count: 18,  Continue Dorado
n 7
Count: 18,  Continue Dorado
n 8
Count: 18,  Continue Dorado
y 9
Count: 19,  Continue Dorado
n 10
Count: 19,  Continue Dorado
n 7
Count: 19,  Continue Dorado
n 8
Count: 19,  Continue Dorado
y 9
Count: 20,  Continue Dorado
n 10
Count: 20,  Continue Dorado
n 7
Count: 20,  Continue Dorado
n 8
Count: 20,  Continue Dorado
y 9
Color done
Count: 21,  Next color
All done!!


## 3 Save raw data - using pickle

**Don't run it**  to avoid overwritting!

For women

In [205]:
#amz1_womentshirt_data=products_pag_dict

#with open('amz1_womentshirt_data.pkl', 'wb') as file:
#    pickle.dump(amz1_womentshirt_data, file)


For men

In [31]:
#amz_men_tshirt_data=products_pag_dict

#with open('amz_men_tshirt_data.pkl', 'wb') as file:
#    pickle.dump(amz_men_tshirt_data, file)